# Counting pairs

In [1]:
import os
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table, unique
from MgIIabs.count import getdata 

In [2]:
data_folder = os.environ['MGIIDATA']
MCdir = "MC data from G Ben Zhu/"
mcfiles = os.listdir(data_folder+MCdir)[:-1]
fullData = getdata.getObsData()

In [3]:
minREW = 1
truncData = fullData[fullData['REW_MGII_2796'] > minREW]

In [4]:
zmin = 0
zmax = float('inf')
fullData = Table.read(data_folder+'Trimmed_SDSS_DR7_107_NonBAL_NonRpt_SDSSRpt.fits')
fullData = fullData[fullData['BAL_FLG']==0]
fullData = fullData[(fullData['ZABS']>=zmin)*(fullData['ZABS']<=zmax)]
#Filtering out associated absorbers (<5000 km/s away)
fullData = fullData[abs(getdata.relVec(fullData['ZABS'],
                    fullData['ZQSO']))>(5000/3e5)]
truncData = fullData[fullData['REW_MGII_2796'] > minREW]
groupedData = truncData.group_by(['PLATE','FIBER','MJD'])
uniquePFMJD = groupedData.groups.keys

In [5]:
mctables = [fits.open(data_folder+MCdir+file, memmap=True)[1].data for file in mcfiles]

In [6]:
zgrid = getdata.getZgrid()
zrange = (zgrid>=zmin)*(zgrid<=zmax)

In [16]:
from itertools import combinations as combos
from itertools import chain

for group in groupedData:
    zlist = group['ZABS']
    pairs = zlist

In [14]:
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()

#final_list = Table(names=('zabs','cover','rewgrid'),dtype=(list,list,list))
final_list = []

for pfmjd, group in zip(uniquePFMJD,groupedData):
    for tab in mctables:
        temp = tab[tab['PLATE']==pfmjd[0]]
        temp = temp[temp['FIBER']==pfmjd[1]]
        if len(temp)==0:
            continue
        else:
            final_list.append([group['ZABS'],temp[0][2][zrange],temp[0][3][zrange]])
            break
pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

KeyboardInterrupt: 

In [17]:
t=Table(names=('zabs','cover'),dtype=(list,list))
#help(Table.add_row)
t.add_row([[1,1],[1,1]])

In [77]:
del final_list

In [ ]:
zlist = [group['ZABS'].data for group in groupedData.groups]

In [94]:
groupedData.groups[0]['ZABS'].data

array([ 1.38195205], dtype=float32)

In [9]:
temp = mctables[0]
temp.columns

ColDefs(
    name = 'PLATE'; format = 'J'
    name = 'FIBER'; format = 'J'
    name = 'ISITCOVERED'; format = '7835B'
    name = 'REWMIN_MGII_2796'; format = '7835E'
    name = 'REWMIN_MGII_2803'; format = '7835E'
)